
#### past
- notebook "2020-06-14.ipynb", tested the capability to continue fitting `clf.fit(X[part], y_enc[part], xgb_model=prev_model)` and saving `clf.save_model(prev_model)` a model in a loop, over parts of a training set. 
- however, the `predict_proba()` step on the test set at each iteration, was taking a very long time. Somtimes half an hour or 40 minutes. 
- In notebook "2020-06-16.ipynb" , I switrched to evaluating on that test set every epoch instead of every batch. Still, this finished `14` epochs in `13` hours and I even cut my data set by `60x` . That also appears to have deteriorated the test accuracy from `acc=0.12` in "2020-06-14.ipynb" to `acc=0.021` in "2020-06-16.ipynb". Again the train and test sets were cut by `60x` so that definitely changed things. But also the `14` epochs did not appear to be helping. 


#### other notes
- feels like better balanced datasets are important . Perhaps otherwise the time to use them is maybe excessively long.
- predict proba is slow
- I still feel like `100` iterations feels like a minimum of sorts to aim for.
- got to re-do 3 or 5 fold cross validation now that weird log loss is good.
- If I do the non-sklearn xgboost, I can take advantage of that dataset caching, which I think saved on memory.
- So perhaps one goal is to try that non-sklearn xgboost, w/ `100` iterations, but using at least those parameters from bundle `'/opt/program/artifacts/2020-06-11T041641Z/bundle.joblib'` which appeared to be a good start..

```

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
               colsample_bynode=1, colsample_bytree=1, gamma=0,
               learning_rate=0.1, max_delta_step=0, max_depth=3,
               min_child_weight=1, missing=nan, n_estimators=100, n_jobs=1,
               nthread=None, objective='multi:softprob', random_state=0,
               reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
               silent=None, subsample=1, verbosity=1),
               
```